## 结构化输出 (Structured Output)

在这个notebook中，我们将演示如何让LLM返回结构化的数据格式，而不是纯文本。我们将使用与chatbot相同的例子：求Normal分布的最大似然估计(MLE)。

### 为什么需要结构化输出？
- 便于程序处理和解析
- 减少格式错误
- 提高数据的可靠性和一致性
- 便于与其他系统集成

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
import dotenv

dotenv.load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [ ]:
import json
from langchain_core.messages import HumanMessage, SystemMessage

system_prompt = """
你是一个数学助手。请用以下JSON格式回答问题：
{
  "answer": "答案",
  "explanation": "推导过程"
}

请确保返回的是有效的JSON格式，不要包含任何额外的文本。
"""

user_question = "Normal分布的最大似然估计(MLE)公式是什么？"

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_question)
]

response = model.invoke(messages)
print("原始回复:")
print(response.content)
print("\n" + "="*50 + "\n")

# 手动解析JSON
try:
    # 尝试解析JSON
    content = str(response.content)
    parsed_result = json.loads(content)
    print("解析成功！")
    print("答案:")
    print(parsed_result["answer"])
    print("\n解释:")
    print(parsed_result["explanation"])
except json.JSONDecodeError as e:
    print(f"JSON解析失败: {e}")
    print("原始内容:")
    print(response.content)


原始回复:
{
  "answer": "对于来自正态分布 $N(\\mu, \\sigma^2)$ 的 $n$ 个独立同分布样本 $x_1, x_2, \\ldots, x_n$，其最大似然估计（MLE）公式如下：\n\n1.  **均值 $\\mu$ 的最大似然估计：**\n    $\\hat{\\mu}_{MLE} = \\frac{1}{n} \\sum_{i=1}^{n} x_i = \\bar{x}$\n    （即样本均值）\n\n2.  **方差 $\\sigma^2$ 的最大似然估计：**\n    $\\hat{\\sigma}^2_{MLE} = \\frac{1}{n} \\sum_{i=1}^{n} (x_i - \\bar{x})^2$\n    （即有偏样本方差）",
  "explanation": "为了推导正态分布参数 $\\mu$ 和 $\\sigma^2$ 的最大似然估计，我们首先写出正态分布的概率密度函数（PDF）：\n$f(x | \\mu, \\sigma^2) = \\frac{1}{\\sqrt{2\\pi\\sigma^2}} e^{-\\frac{(x-\\mu)^2}{2\\sigma^2}}$\n\n给定 $n$ 个独立同分布的样本 $X = (x_1, x_2, \\ldots, x_n)$，似然函数 $L(\\mu, \\sigma^2 | X)$ 是各样本 PDF 的乘积：\n$L(\\mu, \\sigma^2 | X) = \\prod_{i=1}^{n} f(x_i | \\mu, \\sigma^2) = \\prod_{i=1}^{n} \\frac{1}{\\sqrt{2\\pi\\sigma^2}} e^{-\\frac{(x_i-\\mu)^2}{2\\sigma^2}}$\n$L(\\mu, \\sigma^2 | X) = (2\\pi\\sigma^2)^{-n/2} e^{-\\frac{1}{2\\sigma^2} \\sum_{i=1}^{n} (x_i-\\mu)^2}$\n\n为了简化计算，我们通常使用对数似然函数（log-likelihood function），因为对数函数是单调递增的，所以最大化似然函数等价于最大化对数似然函数：\n$\\ln L(\\mu, \\

In [ ]:
from pydantic import BaseModel, Field

class MathResponse(BaseModel):
    answer: str = Field(description="详细的数学推导过程")
    explanation: str = Field(description="简要说明结果的含义")

structured_model = model.with_structured_output(MathResponse)

user_question = "请推导Normal分布的最大似然估计(MLE)公式"

print("使用Pydantic结构化输出:")
result = structured_model.invoke(user_question)

print("类型:", type(result))
print("答案:")
print(result.answer)  # type: ignore
print("\n说明:")
print(result.explanation)  # type: ignore

# 可以直接访问属性，也可以转换为字典
print("\n转换为字典:")
if hasattr(result, 'model_dump'):
    result_dict = result.model_dump()  # type: ignore
else:
    result_dict = result
print(result_dict)


### 为什么不应该自己实现JSON模式？

虽然方法一（手动解析JSON）看起来简单直接，但实际上存在许多问题：

#### 1. 可靠性问题
- **格式不稳定**：LLM可能返回不符合JSON格式的内容
- **解析错误**：手动解析容易出现各种边界情况
- **不一致性**：不同的调用可能返回不同的格式

#### 2. 维护成本
- **错误处理复杂**：需要处理各种解析异常
- **格式验证**：需要手动验证字段是否存在和类型是否正确
- **代码重复**：每个不同的结构都需要重新编写解析逻辑

#### 3. 缺乏类型安全
- **运行时错误**：字段访问错误只能在运行时发现
- **IDE支持差**：无法获得代码补全和类型提示
- **重构困难**：修改数据结构时容易遗漏更新

#### 4. 结构化输出的优势
- **内置验证**：自动验证数据类型和结构
- **类型安全**：编译时检查，IDE支持完整
- **一致性保证**：框架确保输出格式的一致性
- **易于维护**：使用标准化的方式定义和使用数据结构

#### 5. 最佳实践
- **使用框架提供的结构化输出功能**
- **定义清晰的Pydantic模型**
- **利用类型系统提供的安全性**
- **避免手动字符串解析**

### 结论

现代LLM框架（如LangChain）提供的结构化输出功能已经很成熟，可以自动处理JSON Schema生成、验证和解析。我们应该利用这些工具，而不是重新发明轮子。
